# Jax Level 0 Data Scientist Experience - Chapter 3
## Part 2 - New account registration and code execution requests

Link to the original Jax tutorial: https://jax.readthedocs.io/en/latest/jax-101/03-vectorization.html

In [ ]:
# Import the necessary libraries
import syft as sy
sy.requires(">=0.8-beta")
import jax
import jax.numpy as jnp

In [ ]:
# Register a client to the domain
node = sy.orchestra.launch(name="test-domain-1")
guest_domain_client = node.client
guest_domain_client.register(name="Jane Doe", email="jane@caltech.edu", password="abc123", institution="Caltech", website="https://www.caltech.edu/")
guest_domain_client.login(email="jane@caltech.edu", password="abc123")

In [ ]:
# Create a function for code execution
# ATTENTION: ALL LIBRARIES USED SHOULD BE DEFINED INSIDE THE FUNCTION CONTEXT!!!

@sy.syft_function(input_policy=sy.ExactMatch(),
                  output_policy=sy.SingleExecutionExactOutput())
def manual_vectorization():
    import jax
    import jax.numpy as jnp

    x = jnp.arange(5)
    w = jnp.array([2., 3., 4.])

    def convolve(x, w):
        output = []
        for i in range(1, len(x)-1):
            output.append(jnp.dot(x[i-1:i+2], w))
        return jnp.array(output)

    print("Convolve:", convolve(x, w))

    xs = jnp.stack([x, x])
    ws = jnp.stack([w, w])
    
    def manually_batched_convolve(xs, ws):
        output = []
        for i in range(xs.shape[0]):
            output.append(convolve(xs[i], ws[i]))
        return jnp.stack(output)

    print("Naive solution for batched convolve", manually_batched_convolve(xs, ws))


    def manually_vectorized_convolve(xs, ws):
        output = []
        for i in range(1, xs.shape[-1] -1):
            output.append(jnp.sum(xs[:, i-1:i+2] * ws, axis=1))
        return jnp.stack(output, axis=1)

    print("Vectorized solution for batched convolve", manually_vectorized_convolve(xs, ws))
    
@sy.syft_function(input_policy=sy.ExactMatch(),
                  output_policy=sy.SingleExecutionExactOutput())
def automatic_vectorization():
    import jax
    import jax.numpy as jnp
    
    x = jnp.arange(5)
    w = jnp.array([2., 3., 4.])

    def convolve(x, w):
        output = []
        for i in range(1, len(x)-1):
            output.append(jnp.dot(x[i-1:i+2], w))
        return jnp.array(output)

    print("Convolve:", convolve(x, w))

    xs = jnp.stack([x, x])
    ws = jnp.stack([w, w])
    
    auto_batch_convolve = jax.vmap(convolve)

    print("Automatic vectorization", auto_batch_convolve(xs, ws))
    
    auto_batch_convolve_v2 = jax.vmap(convolve, in_axes=1, out_axes=1)

    xst = jnp.transpose(xs)
    wst = jnp.transpose(ws)

    print("Automatic vectorization with axes", auto_batch_convolve_v2(xst, wst))
    
    batch_convolve_v3 = jax.vmap(convolve, in_axes=[0, None])

    print("Automatic vectorization with one ax", batch_convolve_v3(xs, w))
    
    jitted_batch_convolve = jax.jit(auto_batch_convolve)

    print("Jitted Automatic vectorization", jitted_batch_convolve(xs, ws))

In [ ]:
# Test our function locally 
manual_vectorization()
automatic_vectorization()

In [ ]:
# Submit the function for code execution
# guest_domain_client.api.services.code.request_code_execution(manual_vectorization)
guest_domain_client.api.services.code.request_code_execution(automatic_vectorization)

In [ ]:
# new_project = sy.Project(name="My Cool Jax Project")
# proj_desc = """Hi, I want to learn Jax and PySyft"""
# new_project.set_description(proj_desc)
# for code_obj in guest_domain_client.code:
#     new_project.add_request(obj=code_obj, permission=sy.UserCodeStatus.EXECUTE)

In [ ]:
# guest_domain_client.submit_project(new_project)

In [ ]:
# guest_domain_client._api = None
# _ = guest_domain_client.api

In [ ]:
guest_domain_client.api.services.code.automatic_vectorization()

### Go to the Data Owner Notebook for Part 2!

## Part 3 - Downloading the Results

In [ ]:
result = guest_domain_client.api.services.code.automatic_vectorization()

In [ ]:
result.get_result()

In [ ]:
print(result.get_stdout())